In [1]:
import pandas as pd
twitter = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding = 'latin1', header=None)

In [2]:
import re
regexMap={r"<[\w'/'\s]*>": "",r"[\'\"\-]+": "",r"@[\w]+":"",r"http\S+" : ""}
def preprocess(datainput):
    t=datainput
    for regx in regexMap.keys():
        t = re.sub(regx, regexMap[regx], t)
    return t

def labelFilter(label):
    if label == 4:
        return 1
    return label

twitter=twitter[[0,5]]
twitter[5]=twitter[5].apply(preprocess)
twitter[0] = twitter[0].apply(labelFilter)
twitter

,0,5
0,0,"Awww, thats a bummer. You shoulda got Davi..."
1,0,is upset that he cant update his Facebook by t...
2,0,I dived many times for the ball. Managed to s...
3,0,my whole body feels itchy and like its on fire
4,0,"no, its not behaving at all. im mad. why am i..."
...,...,...
1599995,1,Just woke up. Having no school is the best fee...
1599996,1,TheWDB.com Very cool to hear old Walt intervi...
1599997,1,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,Happy 38th Birthday to my boo of alll time!!! ...


In [3]:
training_dataset=twitter.sample(frac=0.7)
test_dataset=twitter.drop(training_dataset.index)

In [4]:
print(len(training_dataset))
print(len(test_dataset))

1120000
480000


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(training_dataset[5].values)

CountVectorizer()

In [6]:
stop_words_vectorizer=CountVectorizer(stop_words='english')
stop_words_vectorizer.fit(training_dataset[5].values)

CountVectorizer(stop_words='english')

In [7]:
x_input=stop_words_vectorizer.transform(training_dataset[5].values)
x_test_input=stop_words_vectorizer.transform(test_dataset[5].values)

In [8]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(x_input,training_dataset[0])

MultinomialNB()

In [9]:
test = pd.read_csv("testdata.manual.2009.06.14.csv", encoding="latin1", header=None)

In [14]:
test = test[[0,5]]
test[5]=test[5].apply(preprocess)
test[0] = test[0].apply(labelFilter)
test = test[test[0] != 2]

test_data, test_labels = test[5], test[0]
test_data = stop_words_vectorizer.transform(test_data)

In [16]:
from sklearn.metrics import classification_report
from sklearn import metrics
y_pred_class = nb.predict(x_test_input)
print(metrics.accuracy_score(test_dataset[0].values, y_pred_class))

test_pred = nb.predict(test_data)
print(classification_report(test_labels, test_pred))


0.7657166666666667
              precision    recall  f1-score   support

           0       0.80      0.81      0.81       177
           1       0.82      0.80      0.81       182

    accuracy                           0.81       359
   macro avg       0.81      0.81      0.81       359
weighted avg       0.81      0.81      0.81       359

